In [159]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import src.config as config

from datetime import datetime

from src.inference import (
    get_feature_store, 
    load_batch_of_features_for_inference,
    load_model_from_registry,
    get_model_predictions,
    )

In [169]:
current_date = datetime.now().date()

data = load_batch_of_features_for_inference(current_date)
display(data.head())
display(data.tail())

2025-03-20 16:32:41,873 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-20 16:32:41,876 INFO: Initializing external client
2025-03-20 16:32:41,876 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-20 16:32:44,951 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1051798
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.81s) 


,datetime,demand,ba_code
1878,2024-03-06,47943.0,AECI
17470,2024-03-07,51039.0,AECI
4528,2024-03-08,52012.0,AECI
7874,2024-03-09,56646.0,AECI
10260,2024-03-10,54399.0,AECI


,datetime,demand,ba_code
25100,2025-03-15,2053.0,WAUW
25157,2025-03-16,1965.0,WAUW
25256,2025-03-17,1946.0,WAUW
25244,2025-03-18,2006.0,WAUW
25207,2025-03-19,2158.0,WAUW


In [170]:
model, preprocessing_pipeline = load_model_from_registry()

2025-03-20 16:33:20,322 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-20 16:33:20,325 INFO: Initializing external client
2025-03-20 16:33:20,325 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-20 16:33:22,055 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1051798


In [172]:
# The features end yesterday
features_end = str(current_date - pd.offsets.Day(1))

predictions = get_model_predictions(
    model=model, 
    preprocessing_pipeline=preprocessing_pipeline, 
    X=data, 
    features_end=features_end,
    )
predictions.head()

,ba_code,predicted_demand,datetime
0,AECI,61723.0,2025-03-20
1,AVA,33361.0,2025-03-20
2,AZPS,67868.0,2025-03-20
3,BANC,44702.0,2025-03-20
4,BPAT,143097.0,2025-03-20


In [ ]:
feature_store = get_feature_store()

feature_group_predictions = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=config.FEATURE_GROUP_MODEL_PREDICTIONS_VERSION,
    description="Predictions generated by the model",
    primary_key=["ba_code", "datetime"],
    event_time="datetime",
)

In [155]:
feature_group_predictions.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1051798/fs/1043525/fg/1420834


Uploading Dataframe: 100.00% |██████████| Rows 53/53 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1051798/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(Job('model_predictions_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)